In [319]:
import pandas as pd
import numpy as np

In [320]:
df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN','londON_StockhOlm','Budapest_PaRis', 'Brussels_londOn'],
'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )',
'12. Air France', '"Swiss Air"']})

1. Some values in the the FlightNumber column are missing. These numbers are
meant to increase by 10 with each row so 10055 and 10075 need to be put in
place. Fill in these missing numbers and make the column an integer column
(instead of a float column).

In [321]:
df['FlightNumber'].isna().sum()

2

In [322]:
df.iloc[1:4:2, 1:2] = [10055,10075]
df['FlightNumber'] = df['FlightNumber'].astype(int)
print(df.dtypes)

From_To         object
FlightNumber     int64
RecentDelays    object
Airline         object
dtype: object


In [323]:
df.head()

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045,"[23, 47]",KLM(!)
1,MAdrid_miLAN,10055,[],<Air France> (12)
2,londON_StockhOlm,10065,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,10075,[13],12. Air France
4,Brussels_londOn,10085,"[67, 32]","""Swiss Air"""


2. The From_To column would be better as two separate columns! Split each
string on the underscore delimiter _ to give a new temporary DataFrame with
the correct values. Assign the correct column names to this temporary
DataFrame.

In [324]:
newdf = pd.DataFrame()
newdf[['From','To']] = ''
newdf.iloc[:,:]

,From,To


In [325]:
newdf.To = df['From_To'].apply(lambda x: x.split('_')[0])
newdf.From = df['From_To'].apply(lambda x: x.split('_')[1])

In [326]:
newdf

,From,To
0,paris,LoNDon
1,miLAN,MAdrid
2,StockhOlm,londON
3,PaRis,Budapest
4,londOn,Brussels


3. Notice how the capitalisation of the city names is all mixed up in this
temporary DataFrame. Standardise the strings so that only the first letter is
uppercase (e.g. "londON" should become "London".)

In [327]:
newdf.From = newdf.From.str.capitalize()
newdf.To = newdf.To.str.capitalize()

In [328]:
newdf

,From,To
0,Paris,London
1,Milan,Madrid
2,Stockholm,London
3,Paris,Budapest
4,London,Brussels


4. Delete the From_To column from df and attach the temporary DataFrame
from the previous questions.

In [329]:
df.drop('From_To', inplace = True, axis=1)

In [330]:
df

,FlightNumber,RecentDelays,Airline
0,10045,"[23, 47]",KLM(!)
1,10055,[],<Air France> (12)
2,10065,"[24, 43, 87]",(British Airways. )
3,10075,[13],12. Air France
4,10085,"[67, 32]","""Swiss Air"""


In [346]:
df= pd.merge(newdf,df,left_index=True, right_index=True)

In [347]:
df

,From,To,FlightNumber,RecentDelays,Airline
0,Paris,London,10045,"[23, 47]",KLM(!)
1,Milan,Madrid,10055,[],<Air France> (12)
2,Stockholm,London,10065,"[24, 43, 87]",(British Airways. )
3,Paris,Budapest,10075,[13],12. Air France
4,London,Brussels,10085,"[67, 32]","""Swiss Air"""


5. In the RecentDelays column, the values have been entered into the
DataFrame as a list. We would like each first value in its own column, each

second value in its own column, and so on. If there isn't an Nth value, the value
should be NaN.

In [442]:
import functools
n = functools.reduce(lambda a,b : a if a > b else b,[len(i) for i in df.RecentDelays]) # calculating max value elements in list of recentdelays
n

3

In [443]:
delays = pd.DataFrame(columns= ['delay_'+str(i) for i in range(1,n+1)],index = range(5))
delays

,delay_1,delay_2,delay_3
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN


In [444]:
delays.columns

Index(['delay_1', 'delay_2', 'delay_3'], dtype='object')

In [445]:
j=0
for i in delays.columns:
    for k in range(5):
        try:
            delays[i].iloc[k] = df.RecentDelays[k][j]
            print(df.RecentDelays[k][j])
        except:
            print('didnt find element')
    j=j+1

23
didnt find element
24
13
67
47
didnt find element
43
didnt find element
32
didnt find element
didnt find element
87
didnt find element
didnt find element


In [446]:
delays

,delay_1,delay_2,delay_3
0,23,47,NaN
1,NaN,NaN,NaN
2,24,43,87
3,13,NaN,NaN
4,67,32,NaN


In [447]:
df= pd.merge(df,delays,left_index=True, right_index=True)

In [449]:
df.drop('RecentDelays', inplace=True, axis =1)

In [450]:
df

,From,To,FlightNumber,Airline,delay_1,delay_2,delay_3
0,Paris,London,10045,KLM(!),23,47,NaN
1,Milan,Madrid,10055,<Air France> (12),NaN,NaN,NaN
2,Stockholm,London,10065,(British Airways. ),24,43,87
3,Paris,Budapest,10075,12. Air France,13,NaN,NaN
4,London,Brussels,10085,"""Swiss Air""",67,32,NaN
